In [32]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [33]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [34]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [35]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [36]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
columns = ['price_change_percentage_24h', 'price_change_percentage_7d', 'price_change_percentage_14d', 'price_change_percentage_30d', 'price_change_percentage_60d']
scaled_data = StandardScaler().fit_transform(df_market_data[columns])


In [37]:
# Create a DataFrame with the scaled data
df_scaled_data = pd.DataFrame(scaled_data, columns=columns)


# Copy the crypto names from the original data
df_scaled_data["coin_id"] = df_market_data.index

# Set the coinid column as index
df_scaled_data= df_scaled_data.set_index("coin_id")


# Display sample data
df_scaled_data.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d
coin_id,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203


---

### Find the Best Value for k Using the Original Data.

In [38]:
# Create a list with the number of k-values from 1 to 11
k= list(range(1,11))

In [39]:
# Create an empty list to store the inertia values
inertia = []
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(df_scaled_data)
    inertia.append(k_model.inertia_)

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list


/Users/renemondragon/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/renemondragon/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/renemondragon/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/renemondragon/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default valu

In [40]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)


In [41]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_curve = df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)
elbow_curve

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 3

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [42]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=3, random_state=0)

In [43]:
# Fit the K-Means model using the scaled data
model.fit(df_scaled_data)

/Users/renemondragon/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=3, random_state=0)

In [44]:
# Predict the clusters to group the cryptocurrencies using the scaled data
clusters_prediction = model.predict(df_scaled_data)


# Print the resulting array of cluster values.
clusters_prediction

array([1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0],
      dtype=int32)

In [45]:
# Create a copy of the DataFrame
df_scaled_data_predictions = df_scaled_data.copy()

In [46]:
# Add a new column to the DataFrame with the predicted clusters
df_scaled_data_predictions["predicted_clusters"] = clusters_prediction


# Display sample data
df_scaled_data_predictions.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,predicted_clusters
coin_id,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,1
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,1
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,0
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,0
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,1


In [47]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
scatter_predictions=df_scaled_data_predictions.hvplot.scatter(
    title= "Price Change 24h vs. 7d",
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="predicted_clusters"
)
scatter_predictions

:NdOverlay   [predicted_clusters]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d)

---

### Optimize Clusters with Principal Component Analysis.

In [48]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [49]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
df_pca = pca.fit_transform(df_scaled_data_predictions)

# View the first five rows of the DataFrame. 
df_pca[:5]

array([[ 0.96229421, -0.67164559, -0.1539862 ],
       [ 0.61973456, -1.09692896,  0.15052345],
       [-0.31433237,  0.55720036, -0.20473643],
       [-0.36760387,  0.37501131, -0.15711165],
       [ 2.16491836, -2.10704479, -0.3262176 ]])

In [50]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pca.explained_variance_ratio_

array([0.48419613, 0.2593091 , 0.16195586])

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 

In [51]:
# Create a new DataFrame with the PCA data.
df_pca_2 = pd.DataFrame(
    df_pca,
    columns =["PCA_1", "PCA_2", "PCA_3"]
)

# Copy the crypto names from the original data
df_pca_2["coin_id"] = df_market_data.index


# Set the coinid column as index
df_pca_2 = df_pca_2.set_index("coin_id")


# Display sample data
df_pca_2.head()

,PCA_1,PCA_2,PCA_3
coin_id,,,
bitcoin,0.962294,-0.671646,-0.153986
ethereum,0.619735,-1.096929,0.150523
tether,-0.314332,0.557200,-0.204736
ripple,-0.367604,0.375011,-0.157112
bitcoin-cash,2.164918,-2.107045,-0.326218


---

### Find the Best Value for k Using the PCA Data

In [52]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))

In [53]:
# Create an empty list to store the inertia values
inertia_2 = []
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(df_pca_2)
    inertia_2.append(k_model.inertia_)

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list


/Users/renemondragon/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/renemondragon/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/renemondragon/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/renemondragon/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default valu

In [54]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_2 = {"k": k, "inertia": inertia_2}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow_2 = pd.DataFrame(elbow_data_2)

In [55]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_curve_2 = df_elbow_2.hvplot.line(
    x="k", 
    y="inertia", 
    title="PCA Elbow Curve", 
    xticks=k
)
elbow_curve_2

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 3


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No, it is the same as the one found on the original data.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [56]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=3, random_state=0)

In [57]:
# Fit the K-Means model using the PCA data
model.fit(df_pca_2)


/Users/renemondragon/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=3, random_state=0)

In [58]:
# Predict the clusters to group the cryptocurrencies using the PCA data
clusters_prediction_2 = model.predict(df_pca_2)

# Print the resulting array of cluster values.
clusters_prediction_2

array([2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0],
      dtype=int32)

In [59]:
# Create a copy of the DataFrame with the PCA data
df_pca_predictions = df_pca_2.copy()


# Add a new column to the DataFrame with the predicted clusters
df_pca_predictions["predicted_clusters"] = clusters_prediction_2


# Display sample data
df_pca_predictions.head()


,PCA_1,PCA_2,PCA_3,predicted_clusters
coin_id,,,,
bitcoin,0.962294,-0.671646,-0.153986,2
ethereum,0.619735,-1.096929,0.150523,2
tether,-0.314332,0.557200,-0.204736,0
ripple,-0.367604,0.375011,-0.157112,0
bitcoin-cash,2.164918,-2.107045,-0.326218,2


In [60]:
# Create a scatter plot using hvPlot by setting 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
scatter_predictions_PCA = df_pca_predictions.hvplot.scatter(
    x="PCA_1",
    y="PCA_2",
    by="predicted_clusters",
    hover_cols=['coin_id']
)

scatter_predictions_PCA

:NdOverlay   [predicted_clusters]
   :Scatter   [PCA_1]   (PCA_2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [61]:
# Composite plot to contrast the Elbow curves
elbow_curve * elbow_curve_2

:Overlay
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [62]:
# Composite plot to contrast the clusters
(scatter_predictions + scatter_predictions_PCA).cols(1)


:Layout
   .NdOverlay.I  :NdOverlay   [predicted_clusters]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d)
   .NdOverlay.II :NdOverlay   [predicted_clusters]
      :Scatter   [PCA_1]   (PCA_2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** 